In [1]:
import time
import datetime
from datetime import timedelta, date
import requests
import json
import csv
import urllib.parse
import pandas as pd
import numpy as np

SN = ["AL2002017080059", "AL2002017100054"]
limittime = datetime.datetime.strptime("2019-07-15", "%Y-%m-%d")

for sn in SN:
    starttime = datetime.datetime.strptime("2019-07-11", "%Y-%m-%d")
    endtime = datetime.datetime.strptime("2019-07-12", "%Y-%m-%d")
    main_api = "http://interface.gtlrenewable.com.au/Alpha/GetData.php?"
    changed_api = "sn={}"
    append_api = "&starttime={}&endtime={}"
    
    while True:
        starttime = starttime + datetime.timedelta(days=1)
        endtime = endtime + datetime.timedelta(days=1)
        #url = URL_pattern.format(starttime,endtime)
        url_loo = main_api+changed_api.format(sn)+append_api.format(starttime.strftime("%Y-%m-%d,%H:%M:%S"),endtime.strftime("%Y-%m-%d,%H:%M:%S"))
        print(url_loo)
        
        try:
            r = requests.get(url_loo,timeout=3)
            r.raise_for_status()
        except requests.exceptions.HTTPError as errh:
            print ("Http Error:",errh)
        except requests.exceptions.ConnectionError as errc:
            print ("Error Connecting:",errc)
        except requests.exceptions.Timeout as errt:
            print ("Timeout Error:",errt)
        except requests.exceptions.RequestException as err:
            print ("OOps: Something Else",err)
                    
        trial_data = requests.get(url_loo).json()

        #save the dataset to .json file and make the data looks nice
        with open('trial2_data.json', 'w', encoding = 'utf-8') as f:
            json.dump(trial_data,f, ensure_ascii=False, indent = 4)
    
        trial_df = pd.read_json('trial2_data.json')
        #algorithm for each value        
        trial_df['PVgen'] = trial_df.Ppv1+trial_df.Ppv2+trial_df.PmeterDc
        trial_df['LOAD'] = trial_df.PrealL1+trial_df.PrealL1+trial_df.PrealL1+trial_df.PmeterL1+trial_df.PmeterL2+trial_df.PmeterL3+trial_df.PmeterDc  
        trial_df['FN'] =trial_df.PmeterL1+trial_df.PmeterL2+trial_df.PmeterL3
        trial_df['FEEDIN'] = np.where(trial_df.FN>0, 0, trial_df['FN'].abs())
        trial_df['GridCon'] = np.where(trial_df.FN<100, 0, trial_df.FN)
        
        df = trial_df[['Sn','UploadTime','PVgen','LOAD','FEEDIN','GridCon']]
        #print(df)
        
        #save the data to CSV
        with open('trial_output_v4.csv','a') as fd:
            df.to_csv(fd, header=False)
                
        #Once the the endtime is greater than limittime, the loop will break
        if endtime >= limittime:
            print('NEXT SN')
            break
            
else:
    print('Loop Done')


http://interface.gtlrenewable.com.au/Alpha/GetData.php?sn=AL2002017080059&starttime=2019-07-12,00:00:00&endtime=2019-07-13,00:00:00
Timeout Error: HTTPConnectionPool(host='interface.gtlrenewable.com.au', port=80): Read timed out. (read timeout=3)
http://interface.gtlrenewable.com.au/Alpha/GetData.php?sn=AL2002017080059&starttime=2019-07-13,00:00:00&endtime=2019-07-14,00:00:00
http://interface.gtlrenewable.com.au/Alpha/GetData.php?sn=AL2002017080059&starttime=2019-07-14,00:00:00&endtime=2019-07-15,00:00:00
NEXT SN
http://interface.gtlrenewable.com.au/Alpha/GetData.php?sn=AL2002017100054&starttime=2019-07-12,00:00:00&endtime=2019-07-13,00:00:00
Timeout Error: HTTPConnectionPool(host='interface.gtlrenewable.com.au', port=80): Read timed out. (read timeout=3)
http://interface.gtlrenewable.com.au/Alpha/GetData.php?sn=AL2002017100054&starttime=2019-07-13,00:00:00&endtime=2019-07-14,00:00:00
Timeout Error: HTTPConnectionPool(host='interface.gtlrenewable.com.au', port=80): Read timed out. (rea